## DSR Mini-Competition 
### Rossman Sales Prediction
#### Team 1: John Enevoldsen, Sara Ghasemi, Mena Nasr

This is a notebook to reproduce the results that are submitted for the competition. Data understanding and exploration and visualisations are not included in this notebook.

In [5]:
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import xgboost
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model

In [6]:
sys.path.insert(0, './modules/')
import cleaning as cln
import feature_eng as feng

## 1. Initial Data Preparation

Read the files and make one data frame to be used for training and cross validation, and one to be used for the final test.

In [11]:
#Read the data from 'train', 'store' and 'holdout' files:

full_df_train = pd.read_csv("./data/train.csv")
full_df_store = pd.read_csv("./data/store.csv")
full_df_holdout = pd.read_csv("./data/holdout.csv")

/Users/sara/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
#Merge the 'train' and 'store' data frames, to be used for training and cross validation:

full_df_train_cv = cln.merge(full_df_train, full_df_store)

In [13]:
#Merge the 'holdout' and 'store' data frames, to be used for final test:

full_df_test = cln.merge(full_df_holdout, full_df_store)

## 2. Data Cleaning